In [1]:
import tarfile

import tarfile
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
pd.set_option('display.max_columns', None)

import plotly.graph_objects as go
from sklearn.decomposition import NMF

import re

from sklearn.decomposition import NMF

## Load all the data 

In [2]:
dataset_all = pq.ParquetDataset("bigd/M148/hitdata7days")

In [3]:
columns = ["cookieid","productlist","ordernumber","post_evar7"]
#df = pd.read_parquet(tn.name,columns=columns)
data_all = dataset_all.read(columns=columns).combine_chunks()

In [4]:
data_all_pd = data_all.to_pandas()

In [5]:
data_all_pd.tail(10)

,cookieid,productlist,ordernumber,post_evar7,visitday
42730139,66766296197672086254571863561089794102,None,None,Search,16
42730140,42336954984504414921774913107250050258,None,None,None,16
42730141,17693983337648543312395385556212515037,None,None,wid,16
42730142,85153173066658247329001202340424923203,None,None,wid,16
42730143,81128382723058624884928245544453957460,None,None,wid,16
42730144,83103452230070488727514707142695403239,;NK0G7;;;;,None,wid,16
42730145,78978313603212467237685845152760013485,None,None,None,16
42730146,38462509364520503943541087382094206104,None,None,wid,16
42730147,53686880511144008613364683438838426866,None,None,Browse,16
42730148,15214929409965487617343822276678408192,None,None,None,16


## Subset stuff 

In [6]:
df_orders = data_all_pd[data_all_pd.ordernumber.notnull()]
df_orders_cookies = df_orders["cookieid"].unique()

In [7]:
df_cookies = df_orders[df_orders["cookieid"].isin(df_orders_cookies)]

In [8]:
df_cookies = df_cookies.drop_duplicates(subset='cookieid', keep="first")
df_cookies.shape

(103138, 5)

## Create DataFrame

In [52]:
df_features = pd.DataFrame(0, index = range(0,len(df_orders_cookies)), columns=["cookieid", "productlist_first", "post_evar7"])

In [53]:
df_features.head(5)

,cookieid,productlist_first,post_evar7
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [54]:
df_orders = data_all_pd[data_all_pd.ordernumber.notnull()]
df_orders_cookies = df_orders["cookieid"].unique()
df_cookies = df_orders[df_orders["cookieid"].isin(df_orders_cookies)]


In [55]:
df_cookies = df_cookies.drop_duplicates(subset='cookieid', keep="first")

In [56]:
df_cookies.shape

(103138, 5)

In [57]:
for i in range(0,len(df_orders_cookies)):
    df_features.loc[i,"cookieid"] = df_orders_cookies[i]
    cookie_ind = df_cookies[df_cookies["cookieid"] == df_orders_cookies[i]]
    
    post_evar7 = cookie_ind.head(1).post_evar7.item()

    if(post_evar7 is None):
        df_features.loc[i,"post_evar7"] = None   
    else: 
        df_features.loc[i,"post_evar7"] = post_evar7
    
    if cookie_ind[cookie_ind["productlist"].str.len()>10].productlist.head(1).empty == True:
        df_features.loc[i,"productlist_first"] = None
    else:
        first_product_ugly = str(cookie_ind[cookie_ind["productlist"].str.len() > 10].productlist.head(1).item())
        # Try
        #first_product_ugly = str(first_product_ugly)
    
        if(first_product_ugly[0:2] == ";N"):
        
            first_product_pre = re.search("=([A-Z])\w*", first_product_ugly)
            if first_product_pre is None:
                df_features.loc[i,"productlist_first"] = None 
            else: 
                first_product = re.findall("=([A-Za-z]+)", first_product_pre[0])[0]
                df_features.loc[i,"productlist_first"] = first_product
        else: 
            first_product = re.search("([\w &]*)", first_product_ugly)[0]
            df_features.loc[i,"productlist_first"] = first_product


In [58]:
df_features.head(10)

,cookieid,productlist_first,post_evar7
0,52882151623272742032948034078925051832,Video Games,NaN
1,79755164678756084974501978337153484560,Toys,Browse
2,5443955122341888424003495602170301473,Bed & Bath,Browse
3,15567279697543469666445773480724187157,Personal Care,Search
4,71667249622638493801005527041676393041,Shoes,Search
5,61042077885887970553598772945168024836,Toys,Search
6,50173036520586681086289374017581049714,Kitchen,Browse
7,75806096112973408034688549910937655265,Toys,Search
8,69355983623503946334291902547906312305,Home,None
9,32066281582833015954444801797185030134,Bed & Bath,Browse


In [59]:
#df_features.to_csv('productlist_new.csv')